This workbook loads and converts a SCANCO .ISQ or .AIM file in the directory to several 
.nrrd formatted images with the following units: density (g/cc), Hounsfield Units, and a
binary thresholded image mask 

Created by: Jason Cox, 04/07/2023

Citations: McCormick, Matthew, et al. "ITK: Enabling Reproducible 
Research and Open Science.” Frontiers in Neuroinformatics, vol. 8, 
Frontiers Media SA, 2014, doi:10.3389/fninf.2014.00013.

In [20]:
import numpy as np
import cv2
import re
import itk
import glob

In [2]:
# ISQ and AIM metadata returned by dict (cell 3) contain the same fields. Therefore, this program could be modified to loop over all files
# in the directory, if that is the desired implementation. The variable all_filenames is unused in the current version, but can be used in a
# modified version that implements a loop.

all_AIM_filenames = glob.glob("*.AIM")
all_ISQ_filenames = glob.glob("*.ISQ")
all_filenames = all_AIM_filenames + all_ISQ_filenames

del all_AIM_filenames, all_ISQ_filenames


In [22]:
filename = 'C0005757_SCAP.AIM'
image = itk.imread(filename)
metadata = dict(image)
HUpixels = itk.array_view_from_image(image)
del image
# HUpixels = np.array(HUpixels)                     # Array arithmetic below seems to work without this line, but certain operations were not tested as they require too much memory

In [ ]:
# As discussed below, some required constants are missing from metadata. Compare the output of this cell with the file header at:
# https://docs.google.com/document/d/1L2GCnHa4jmK87p0pGI9MNN10MWRMwrk2WqPbPeklP_I/edit

print(metadata)

The next cell provides a template file header from C0005757_SCAP.AIM, but this needs to be read from the .AIM or .ISQ.

Then, the following cell identifies numeric constants in the header necessary for converting the image in HU (units given by itk.imread)
to density, or for thresholding/masking the image.

The only constants not readable from the header which must be provided by the user are the per-milli values for low and high thresholds. In basic terms, the low/high threshold pixel values are computed as the respective low/high per-milli value multiplied by the highest pixel value in the image (a quanitity that is given by the header).

In [11]:
header = '{Global Density: intercept=-1.96600006e+02, Global Calib. default unit type=2 (Density), Global Position Slice 1 [um]=47050, Global HU: mu water=0.51400, Global Reconstruction-Alg.=3, Global Parameter units=[1/cm], Global Average data value=322.51215, Global Minimum data value=-2891.00000, Global Created by=ISQ_TO_AIM (IPL), Global Patient Name=JC-NBPI-Rat-May17, Global Energy [V]=70000, Global Scanner ID=2135, Global Standard deviation=0.32443, Global Time=29-JUN-2022 12:39:11.75, Global Orig-ISQ-Dim-p=1792       1792       1781, Global Original Creation-Date=20-AUG-2018 23:18:59.99, Global Parameter name=Linear Attenuation, Global Orig-ISQ-Dim-um=17918      17918      17808, Global Scanner type=10, Global Scan Distance [um]=20480, Global Index Patient=136, Global Index Measurement=7866, Global Minimum value=-0.70581, Global Maximum data value=32767.00000, Global Intensity [uA]=114, Global Maximum value=7.99976, Global Density: slope=3.55519989e+02, Global Mu_Scaling=4096, Global Reference line [um]=47050, Global No. samples=2048, Global Default-Eval=15, Global Site=4, Global Average value=0.07874, Global Calibration Data=Unknown. Backconverted from AIM., Global Standard data deviation=1328.87610, Global No. projections per 180=1000, Global Original file=DK0:[MICROCT.DATA.00000136.00007866]B0005757.ISQ, Global Scaled by factor=4096, Global Angle-Offset [mdeg]=0, Global Integration time [us]=800000, Global Density: unit=mg HA/ccm}'

In [16]:
# Gather numeric constants needed for unit conversion. Currently, the file metadata returned with dict does not contain all of the
# necessary constants. For an example of a full file header with highlighted variables of interest, see the google doc 
# "SCANCO micro-CT Units Conversion" at: https://docs.google.com/document/d/1L2GCnHa4jmK87p0pGI9MNN10MWRMwrk2WqPbPeklP_I/edit

mu_threshLow = 0.250
mu_threshHigh = 1.0

scancoMU = float(re.search('Global Scaled by factor=(.*?),', header).group(1))
# scancoMU = metadata['MuScaling']
maxVal = float(re.search('Global Maximum data value=(.*?),', header).group(1))
# maxVal = metadata['**********']
densitySlope = float(re.search('slope=(.*?),', header).group(1))
# densitySlope = metadata['**********']
densityIntercept = float(re.search('intercept=(.*?),', header).group(1))
# densityIntercept = metadata['**********']
water_mu = float(re.search('water=(.*?),', header).group(1))
# water_mu = metadata['MuWater']

del header                  # when the code base is modified such that metadata gives all relevant constants, this line and others that reference "header" can be deleted
del metadata

muLow = mu_threshLow * maxVal
mu_pixelLow = muLow/scancoMU
muHigh = mu_threshHigh * maxVal
mu_pixelHigh = muHigh/scancoMU
HU_pixelLow = (-1000) + mu_pixelLow * (1000/water_mu)
HU_pixelHigh = (-1000) + mu_pixelHigh * (1000/water_mu)
HUvalues = [HU_pixelLow, HU_pixelHigh]

In [7]:
# Create unit-converted outputs. MUpixels is not desired as an output due to representing non-standard units. However, creation of MUpixels
# can be skipped, as RHOpixels can be created by combining the functions needed to convert HU to MU and MU to RHO.

# MUpixels = (HUpixels+1000)/(1000/water_mu)
# RHOpixels = (MUpixels*densitySlope) + densityIntercept
RHOpixels = ((((HUpixels+1000)/(1000/water_mu))*densitySlope) + densityIntercept)/1000  # Dividing by 1000 at the end changes units from mg/cc to g/cc, which is more commonly published
temp = HUpixels.copy()
temp[temp>HUvalues[1]] = 0
BWmask = cv2.threshold(temp, HUvalues[0], 1, cv2.THRESH_BINARY)[1]

In [8]:
# Create .nrrd file output with units of density (mg/cc or g/cc)

RHOout = itk.image_view_from_array(RHOpixels)
RHOoutname = filename[0:-4] + '_RHO.nrrd'
itk.imwrite(RHOout, RHOoutname)

In [9]:
# Create a binary image mask .nrrd file output based on user-provided low and high threshold values.

BWout = itk.image_view_from_array(BWmask)
BWoutname = filename[0:-4] + '_BWmask.nrrd'
itk.imwrite(RHOout, BWoutname)


In [6]:
# Create .nrrd file output with Hounsfield Units (equivalent to loading the file in 3D Slicer)

HUout = itk.image_view_from_array(HUpixels)
HUoutname = filename[0:-4] + '_HU.nrrd'
itk.imwrite(HUout, HUoutname)